# Product-mix Pyomo

$$
\begin{align}
    \text{max} \quad & \sum_{j \in J} c_j x_j \\
    \text{s.t.} \quad & \sum_{j \in J} a_{i, j} x_{j} \leq b_{i} & \forall \; i \in I \\
    & x_{j} \geq 0 & \forall \; j \in J \\
\end{align}
$$

In [1]:
# Python native modules
import json

# Third-party packages
import pyomo.environ as pyo

In [2]:
# Read input file and store in local variable `input_data`
with open("input_prod_mix.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [3]:
input_data

{'margins': [{'product': 'P1', 'value': 2.15},
  {'product': 'P2', 'value': 1.34},
  {'product': 'P3', 'value': 1.72}],
 'availabilities': [{'resource': 'R1', 'value': 80},
  {'resource': 'R2', 'value': 30},
  {'resource': 'R3', 'value': 25}],
 'proportions': [{'resource': 'R1', 'product': 'P1', 'proportion': 0.7},
  {'resource': 'R1', 'product': 'P2', 'proportion': 0.3333333333333333},
  {'resource': 'R1', 'product': 'P3', 'proportion': 0.5},
  {'resource': 'R2', 'product': 'P1', 'proportion': 0.2},
  {'resource': 'R2', 'product': 'P2', 'proportion': 0.6666666666666666},
  {'resource': 'R2', 'product': 'P3', 'proportion': 0.16666666666666666},
  {'resource': 'R3', 'product': 'P1', 'proportion': 0.1},
  {'resource': 'R3', 'product': 'P2', 'proportion': 0.0},
  {'resource': 'R3', 'product': 'P3', 'proportion': 0.3333333333333333}]}

In [4]:
availabilities = {b["resource"]: b["value"] for b in input_data["availabilities"]}
margins = {c["product"]: c["value"] for c in input_data["margins"]}
proportions = {
    (p["resource"], p["product"]): p["proportion"]
    for p in input_data["proportions"]
}

In [6]:
print(availabilities)
print('\n')
print(margins)
print('\n')
print(proportions)

{'R1': 80, 'R2': 30, 'R3': 25}


{'P1': 2.15, 'P2': 1.34, 'P3': 1.72}


{('R1', 'P1'): 0.7, ('R1', 'P2'): 0.3333333333333333, ('R1', 'P3'): 0.5, ('R2', 'P1'): 0.2, ('R2', 'P2'): 0.6666666666666666, ('R2', 'P3'): 0.16666666666666666, ('R3', 'P1'): 0.1, ('R3', 'P2'): 0.0, ('R3', 'P3'): 0.3333333333333333}


In [7]:
# Create model instance
model = pyo.ConcreteModel()

In [9]:
# Sets for resources I and products J
model.I = pyo.Set(initialize=availabilities.keys())
model.J = pyo.Set(initialize=margins.keys())

In [10]:
# Paramters
model.c = pyo.Param(model.J, initialize=margins)
model.b = pyo.Param(model.I, initialize=availabilities)
model.a = pyo.Param(model.I, model.J, initialize=proportions)

In [13]:
model.a.display()

a : Size=9, Index=I*J, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('R1', 'P1') :                 0.7
    ('R1', 'P2') :  0.3333333333333333
    ('R1', 'P3') :                 0.5
    ('R2', 'P1') :                 0.2
    ('R2', 'P2') :  0.6666666666666666
    ('R2', 'P3') : 0.16666666666666666
    ('R3', 'P1') :                 0.1
    ('R3', 'P2') :                 0.0
    ('R3', 'P3') :  0.3333333333333333


In [14]:
# Decision variables
model.x = pyo.Var(model.J, within=pyo.NonNegativeReals)

In [15]:
# Resource availablity constraints
def av_cstr(model, i):
    return sum(model.a[i, j] * model.x[j] for j in model.J) <= model.b[i]


model.av_cstr = pyo.Constraint(model.I, rule=av_cstr)

In [16]:
model.av_cstr.pprint()

av_cstr : Size=3, Index=I, Active=True
    Key : Lower : Body                                                             : Upper : Active
     R1 :  -Inf :                 0.7*x[P1] + 0.3333333333333333*x[P2] + 0.5*x[P3] :  80.0 :   True
     R2 :  -Inf : 0.2*x[P1] + 0.6666666666666666*x[P2] + 0.16666666666666666*x[P3] :  30.0 :   True
     R3 :  -Inf :                 0.1*x[P1] + 0.0*x[P2] + 0.3333333333333333*x[P3] :  25.0 :   True


In [17]:
# Objective function
def obj(model):
    return sum(model.c[j] * model.x[j] for j in model.J)



model.obj = pyo.Objective(rule=obj, sense=pyo.maximize)

In [19]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 2.15*x[P1] + 1.34*x[P2] + 1.72*x[P3]


In [18]:
# Instantiate Highs persistent solver (make sure highspy is installed)
solver = pyo.SolverFactory("appsi_highs")

In [20]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 258.8526315789474, 'Upper bound': 258.8526315789474, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [21]:
# Use objective as a callable to see its value
model.obj()

258.8526315789474

In [22]:
# Print results
for j, xi in model.x.extract_values().items():
    print(f"{xi:.2f} units of Product {j}")

71.05 units of Product P1
10.26 units of Product P2
53.68 units of Product P3
